In [1097]:
import numpy as np
import pandas as pd
from daty_transakcji import *
from osoby import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [1098]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "PESEL",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [1099]:
employees_df

,id,imię,nazwisko,PESEL,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Kazimierz,Opałka,64012501434,Wrocław,ul. Wspólna 9/32,48747679678,kazimierz.opalka@kuba.pl,2014-01-01,None,menadżer,69.5
1,2,Genowefa,Nefediuk,92021653402,Wrocław,ul. rtm. Zbigniewa Dunin-Wąsowicza 89/35,48638163323,genowefa.nefediuk@kuba.pl,2014-01-01,None,sprzedawca,45.0
2,3,Małgorzata,Selina,92041845285,Pieszyce,ul. Janisława Grondysa 49/7,48782294598,malgorzata.selina@kuba.pl,2014-01-01,None,sprzedawca,47.5
4,4,Julian,Kokoulin,62102027491,Wrocław,ul. Wesoła 25,48530214480,julian.kokoulin@kuba.pl,2014-01-01,2016-11-09,mechanik,55.0
11,5,Andrei,Durło,68051549352,Kołobrzeg,ul. Miechowicka 7/21,48495462869,andrei.durlo@kuba.pl,2014-01-01,None,informatyk,45.3
3,6,Maciej,Słowiński,90032171230,Wrocław,ul. Wiejska 8/48,48940262319,maciej.slowinski@kuba.pl,2015-06-25,2016-12-27,sprzedawca,51.8
5,7,Yury,Wesołowski,70120128672,Wrocław,ul. Jesionowa 4,48242422052,yury.wesolowski@kuba.pl,2015-12-04,None,mechanik,48.5
6,8,Monika,Iskrzycka,76012587666,Wrocław,ul. Poprzeczna 39/33,48615203987,monika.iskrzycka@kuba.pl,2016-04-14,None,mechanik,50.8
7,9,Ignacy,Chojnacki,68020255778,Wrocław,ul. Jęczmienna 84,48403925943,ignacy.chojnacki@kuba.pl,2016-08-06,None,pomocnik mechanika,41.4
8,10,Andrzej,Zalewski,89032285733,Wrocław,ul. Stanisława Chudoby 71/18,48672480613,andrzej.zalewski@kuba.pl,2016-09-10,None,diagnosta samochodowy,58.2


In [1100]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [1101]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [1102]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [1103]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4])-19 for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()
not_fixed_our_cars = our_cars[:]

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if len(not_fixed_our_cars) >= 1 and np.random.rand() <= weights[1]/2: #fixing car we currently have
           client_ids[i] = 0
           car_ids[i] = np.random.choice(not_fixed_our_cars)
           not_fixed_our_cars.remove(car_ids[i])
        elif np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [1104]:
len(year_ceil), len(pd.unique(client_ids))

(1405, 1406)

In [1105]:
year_ceil

[1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [1106]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [1107]:
len(buy_indexes[0]), len(sell_indexes[0]), len(service_indexes[0])

(760, 756, 461)

In [1108]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [1109]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [1110]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [1111]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [1112]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])

menager = employees_df.iloc[0]
if int(menager["PESEL"][-2]) % 2 == 1:
    menager_sex = "M"
else:
    menager_sex = "K"
clients_df.loc[0] = [0, menager["imię"], menager["nazwisko"], menager_sex, menager["PESEL"], menager["miasto"], menager["adres"], False, menager["telefon"], menager["email"]]

unique_clients = np.unique(client_ids)
unique_clients = unique_clients[unique_clients != 0]
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Kazimierz,Opałka,M,64012501434,Wrocław,ul. Wspólna 9/32,False,48747679678,kazimierz.opalka@kuba.pl
1,1,Honoriusz,Kończakowski,M,86092643572,Wrocław,ul. Zbożowa 17/45,False,48398384888,h-konczakowski86@outlook.com
2,2,Iryna,Wąsik,K,84031733874,Dąbie,ul. Starcówka 84,True,48518958248,i_wasik84@gmail.com
3,3,Natalia,Piekarz,K,93022218335,Zielona Góra,ul. Boczna 47/11,False,48523822003,piekarz19935@protonmail.com
4,4,Kateryna,Kos,K,85031808218,Stargard,ul. Żytnia 62,False,48345980953,kos5@interia.pl
...,...,...,...,...,...,...,...,...,...,...
1401,1401,Maria,Petričević,K,62032804238,Ustka,ul. Gajowa 96,False,48458169090,petricevic19626@gmail.com
1402,1402,Alla,Khodis,K,82120184792,Wrocław,ul. Wiejska 80,False,48640568434,all_kho@interia.pl
1403,1403,Mateusz,Dahms,M,83120766872,Wrocław,ul. Spokojna 34/22,False,48904767556,m.dahms@onet.pl
1404,1404,Bartosz,Skóra,M,82040918431,Rybnik,ul. Długa 32/6,False,48754579090,bsko@onet.pl


In [1113]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [1114]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Kazimierz,Opałka,M,64012501434,Wrocław,ul. Wspólna 9/32,False,48747679678,kazimierz.opalka@kuba.pl
1,1,Honoriusz,Kończakowski,M,86092643572,Wrocław,ul. Zbożowa 17/45,False,48398384888,h-konczakowski86@outlook.com
2,2,Iryna,Wąsik,K,84031733874,Dąbie,ul. Starcówka 84,True,48518958248,i_wasik84@gmail.com
3,3,Natalia,Piekarz,K,93022218335,Zielona Góra,ul. Boczna 47/11,False,48523822003,piekarz19935@protonmail.com
4,4,Kateryna,Kos,K,85031808218,Stargard,ul. Żytnia 62,False,48345980953,kos5@interia.pl
...,...,...,...,...,...,...,...,...,...,...
1401,1401,Maria,Petričević,K,62032804238,Ustka,ul. Gajowa 96,False,48458169090,petricevic19626@gmail.com
1402,1402,Alla,Khodis,K,82120184792,Wrocław,ul. Wiejska 80,False,48640568434,all_kho@interia.pl
1403,1403,Mateusz,Dahms,M,83120766872,Wrocław,ul. Spokojna 34/22,False,48904767556,m.dahms@onet.pl
1404,1404,Bartosz,Skóra,M,82040918431,Rybnik,ul. Długa 32/6,False,48754579090,bsko@onet.pl


In [1115]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [1116]:
np.sum(clients_df['karta_lojalnościowa'])

89

In [1117]:
np.sum(pd.isna(clients_df["PESEL"]))

198

# auta

In [1118]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [1119]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [1120]:
df_samochody = create_car_table(max(car_ids))

In [1121]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Audi,RS3,4,2004,Kombi,False,szary,Benzyna,6.4,400,Automatyczna,5,5,False
1,2,Volkswagen,Golf,4,2012,SUV,False,biały,Benzyna,1.7,105,Manualna,5,2,False
2,3,Land,Rover Range Rover Velar,4,2011,SUV,False,czarny,Diesel,4.0,250,Automatyczna,5,5,False
3,4,Volkswagen,Tiguan,4,2017,Kombi,False,szary,Benzyna,2.5,150,Manualna,5,5,False
4,5,Toyota,Aygo,4,2008,Sedan,False,czarny,Benzyna,1.1,68,Manualna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026,1027,BMW,X1,4,2014,Kombi,True,biały,Benzyna,3.2,192,Automatyczna,5,5,False
1027,1028,Opel,Mokka,4,2006,Sedan,False,szary,Benzyna,2.0,131,Manualna,5,2,False
1028,1029,Skoda,Fabia,4,2011,Van,False,srebrny,Benzyna,1.4,90,Manualna,5,5,True
1029,1030,Fiat,500,4,2006,Hatchback,False,szary,Benzyna,1.2,69,Manualna,5,5,True


In [1122]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [1123]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Automatyczna', 'Manualna', 'Półautomatyczna'], dtype=object)

# Usługi

In [1124]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [1125]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [1126]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [1127]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2007' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [1128]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [1129]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,3,2014-01-02,126200.0
1,2,2,2,3,2014-01-02,72700.0
2,3,3,3,6,2014-01-03,111700.0
3,4,4,4,3,2014-01-04,60900.0
4,5,5,5,2,2014-01-04,29300.0
...,...,...,...,...,...,...
755,756,1025,1396,2,2017-12-22,38600.0
756,757,1028,1401,3,2017-12-27,37200.0
757,758,1029,1402,6,2017-12-27,31600.0
758,759,1030,1019,6,2017-12-27,20900.0


In [1130]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [1131]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [1132]:
loyalty_card_customer_ids = [el[0] for el in loyalty_card_dates]
loyalty_card_dates_sep = [el[1] for el in loyalty_card_dates]

In [1133]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:
        date_index = loyalty_card_customer_ids.index(customer_id)
        if loyalty_card_dates_sep[date_index] < df_sell.loc()[i]['data_zaksięgowania']:
            sell_prices[i] *= 0.97
            #print(i)

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
#print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\1822833189.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:


In [1134]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,13,21,3,85280.0,12,2014-01-15
1,2,10,22,6,93738.0,6,2014-01-17
2,3,20,23,2,99958.0,18,2014-01-17
3,4,11,24,6,29900.0,1,2014-01-17
4,5,9,25,3,69768.0,6,2014-01-18
...,...,...,...,...,...,...,...
751,752,1017,1399,3,23868.0,6,2017-12-27
752,753,1020,1400,6,17000.0,1,2017-12-27
753,754,1021,1403,3,162282.0,6,2017-12-30
754,755,1028,1404,2,49920.0,12,2017-12-30


In [1135]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

0

In [1136]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [1137]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(21, array([11, 19], dtype=int64)), (22, array([9], dtype=int64)), (23, array([32], dtype=int64)), (28, array([49, 56, 16], dtype=int64)), (30, array([63], dtype=int64)), (31, array([63, 49], dtype=int64)), (32, array([ 5, 29], dtype=int64)), (33, array([14, 21, 45], dtype=int64)), (35, array([14, 39,  9], dtype=int64)), (39, array([53], dtype=int64)), (42, array([40], dtype=int64)), (44, array([30], dtype=int64)), (49, array([29], dtype=int64)), (52, array([36], dtype=int64)), (54, array([13, 60], dtype=int64)), (55, array([34], dtype=int64)), (56, array([49, 41, 52], dtype=int64)), (65, array([0], dtype=int64)), (69, array([33], dtype=int64)), (70, array([60], dtype=int64)), (74, array([40, 13, 40], dtype=int64)), (77, array([36], dtype=int64)), (78, array([59], dtype=int64)), (80, array([5], dtype=int64)), (81, array([29, 20], dtype=int64)), (82, array([63], dtype=int64)), (83, array([22, 28], dtype=int64)), (86, array([10,  4], dtype=int64)), (89, array([61, 22, 54], dtype=int64))

In [1138]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [1139]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Drzwi prawe tylne,Citroen C1 szary,308,1,1
1,Dystrybutor,Citroen C1,193,1,1
2,Drzwi prawe przednie,Renault Clio biały,280,2,1
3,Pasy bezpieczeństwa,,50,3,1
4,Filtr paliwa,SEAT Altea XL,434,4,1
...,...,...,...,...,...
696,Kolektor wydechowy,,500,459,1
697,Dźwignia zmiany biegów,Opel Zafira Tourer,112,460,1
698,Pompa paliwowa,BMW X1,755,461,1
699,Drzwi lewe tylne,BMW X1 biały,998,461,1


# Wyposażenie

In [1140]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [1141]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Drzwi prawe tylne,Citroen C1 szary,308.0
1,2,Dystrybutor,Citroen C1,193.0
2,3,Drzwi prawe przednie,Renault Clio biały,280.0
3,4,Pasy bezpieczeństwa,,50.0
4,5,Filtr paliwa,SEAT Altea XL,434.0
...,...,...,...,...
694,500,Drzwi prawe przednie,Nissan Micra inny,335.0
695,501,Deska rozdzielcza,Mazda 6,206.0
697,502,Dźwignia zmiany biegów,Opel Zafira Tourer,112.0
698,503,Pompa paliwowa,BMW X1,755.0


# Użyte części - uzupełnienie

In [1142]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df['cena'] = pd.to_numeric(used_parts_df['cena'])

for i in range(len(used_parts_df)):
    used_parts_df.loc()[i, 'cena'] *= rng.uniform(0.9, 1.1)
    used_parts_df.loc()[i, 'cena'] = np.round(used_parts_df.loc()[i, 'cena'], 0)

used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość', 'cena']]

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\1878107147.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '308.5233419239297' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  used_parts_df.loc()[i, 'cena'] *= rng.uniform(0.9, 1.1)


In [1143]:
used_parts_df

,id,id_usługi,id_części,ilość,cena
0,1,1,1,1,309.0
1,2,1,2,1,177.0
2,3,2,3,1,297.0
3,4,3,4,1,46.0
4,5,4,5,1,470.0
...,...,...,...,...,...
696,697,459,135,1,531.0
697,698,460,502,1,118.0
698,699,461,503,1,769.0
699,700,461,504,1,899.0


In [1144]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [1145]:
# def get_service_date(part_id):
    
#     index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
#     service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
#     return service_date
    

# buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
# parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

# for i in range(len(buy_parts_quantity)):
#     part_id = buy_parts_quantity['id_części'][i]
#     quantity = buy_parts_quantity['ilość'][i]
#     price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
#     total_cost = np.floor(price * rng.uniform(0.75, 0.95))[0]
#     service_date = get_service_date(part_id)
    
#     row = [int(i+1), int(part_id), total_cost, int(quantity)]
#     parts_purchase_pd.loc[i] = row


In [1146]:
# parts_purchase_pd

In [1147]:
# parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [1148]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [1149]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi uzupełnienie

In [1150]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

for i in range(len(df_services)):
    if df_services.loc()[i]['id_klienta'] == 0:
        df_services.loc[i, 'cena'] = 0
    else:
        df_services.loc[i, 'cena'] = np.round(service_prices[i] * (rng.uniform(1.25, 1.7)), -1).astype(int)

In [1151]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,21,40,8,2014-01-28,750.0
1,2,22,42,4,2014-01-30,470.0
2,3,23,43,8,2014-01-30,70.0
3,4,28,51,4,2014-02-06,930.0
4,5,30,56,4,2014-02-11,50.0
...,...,...,...,...,...,...
456,457,1012,0,8,2017-12-13,0.0
457,458,1015,0,7,2017-12-17,0.0
458,459,596,897,7,2017-12-20,800.0
459,460,1026,1267,4,2017-12-26,150.0


In [1152]:
df_services.to_csv("outputs/usługi.csv", index=False)